In [1]:
from common_functions import *
from game import CCGame
from state import StateSpace
from rewarder import Rewarder
from DQN import DQNAgent

plt.rcParams['axes.facecolor'] = '#323A48'
plt.rcParams['axes.edgecolor'] = '#92A2BD'
plt.rcParams['figure.facecolor'] = '#323A48'
plt.rcParams['text.color'] = '#DBE1EA'
plt.rcParams['xtick.color'] = '#DBE1EA'
plt.rcParams['ytick.color'] = '#DBE1EA'
plt.rcParams['grid.linestyle'] = ':'
plt.rcParams['grid.color'] = '#3F495A'

from cycler import cycler
plt.rcParams['axes.prop_cycle'] = cycler(color=['c', 'goldenrod', 'mediumseagreen', 'blueviolet', 'indianred', 'mediumpurple', 'cornflowerblue',
                                               'darkorchid', 'gold', 'olivedrab'])

In [2]:
rewarder = Rewarder()
agent = DQNAgent()
stateSpace = StateSpace()

def testGameTime():
  g = CCGame()
  g.start()
  while not g.crashed:
    state = stateSpace.stateFromTick(g.car_id, g.tick_data, g.mapMatrix)
    command = g.send_random_command()
    reward = rewarder.calculate_reward(g)
    state_next = stateSpace.stateFromTick(g.car_id, g.tick_data, g.mapMatrix)
    agent.remember(state, command, reward, state_next, done=g.crashed)
  print_t("ticks: {}".format(g.ticknum))
  g.close()
  return
testGameTime()

[2019-04-21 21:54:14] CAR CRASHED.
[2019-04-21 21:54:14] ticks: 13
[2019-04-21 21:54:14] Game already closed.


In [3]:
agent.train_on_memory()

In [5]:
def train_agent():
  rewarder = Rewarder()
  agent = DQNAgent()
  stateSpace = StateSpace()
  game = CCGame()
  
  
  return

In [4]:
g = CCGame()

for k in range(10):
  g.start()
  g.send_random_command()
  g.close()

In [4]:
## Test section
g = CCGame(log_ticks=False)
ss = StateSpace()
rewarder = Rewarder()
g.start()
state = ss.stateFromTick(g.car_id, g.tick_data, g.mapMatrix)
c=g.send_random_command()
reward = rewarder.calculate_reward(g)
g.close()

print_t("Game id: {}".format(g.game_id))
print_t("command: {}".format(c))
print_t("reward: {}".format(reward))

[2019-04-21 21:44:02] Game id: 24985
[2019-04-21 21:44:02] command: CAR_INDEX_RIGHT
[2019-04-21 21:44:02] reward: 0


In [5]:
# import json
# logdir = '/Users/nagydaniel/Documents/Munka/asseco/coding_cup/logs/'
# log_9327 = open(logdir + 'log_9327').read()
# log_9327 = json.loads(log_9327)
# log_9327[0]

In [6]:
# ss = StateSpace()
# state = ss.stateFromTick(carid, log_9327[0]['thick'], g.mapMatrix)
# state

In [7]:
len(agent.memory)

75

In [8]:
""" TODO: As a baseline project, go to a certain point without dying. Use networkx
to find the shortest path to the point.
"""

' TODO: As a baseline project, go to a certain point without dying. Use networkx\nto find the shortest path to the point.\n'

In [9]:
a=[{"id": 2, "life": 100}, {"id": 0, "life":80}]
list(filter(lambda x: x["id"]==0, a))

[{'id': 0, 'life': 80}]

In [10]:
jsonstr = '[{"x":-1,"y":-7},{"x":0,"y":-7},{"x":1,"y":-7},{"x":-1,"y":-6},{"x":0,"y":-6},{"x":1,"y":-6},{"x":-1,"y":-5},{"x":0,"y":-5},{"x":1,"y":-5},{"x":-1,"y":-4},{"x":0,"y":-4},{"x":1,"y":-4},{"x":-2,"y":-3},{"x":-1,"y":-3},{"x":0,"y":-3},{"x":1,"y":-3},{"x":2,"y":-3},{"x":-3,"y":-2},{"x":-2,"y":-2},{"x":-1,"y":-2},{"x":0,"y":-2},{"x":1,"y":-2},{"x":2,"y":-2},{"x":3,"y":-2},{"x":-3,"y":-1},{"x":-2,"y":-1},{"x":-1,"y":-1},{"x":0,"y":-1},{"x":1,"y":-1},{"x":2,"y":-1},{"x":3,"y":-1},{"x":-3,"y":0},{"x":-2,"y":0},{"x":-1,"y":0},{"x":1,"y":0},{"x":2,"y":0},{"x":3,"y":0},{"x":-2,"y":1},{"x":-1,"y":1},{"x":0,"y":1},{"x":1,"y":1},{"x":2,"y":1},{"x":-1,"y":2},{"x":0,"y":2},{"x":1,"y":2},{"x":0,"y":3}]'

In [11]:
data = json.loads(jsonstr)

NameError: name 'json' is not defined

In [ ]:
[data[k]['x'] == ss.viewAreaCoords[k][0] and data[k]['y']==ss.viewAreaCoords[k][1] for k in range(len(ss.viewAreaCoords))]

In [ ]:
np.round([0.991, 0.0001]).astype(int)

In [ ]:
#x = {"status": False, "car": None}
x = {"status": True, "car": {"direction": "UP"}}

0 if x['status']==False else x['car']['direction']

In [ ]:
o={"a": {"b": 2}}

In [ ]:
o["a"]["b"]

In [ ]:
a = {"p": 1, "q": 3}
b = a 
a = {"p":2, "q": 2}

In [ ]:
b

In [ ]:
a = ['END REASON: CRASHED', 'jsadkasjdkl']
'crash' if 'END REASON: CRASHED' in a else 'no crash'

In [ ]:
a = {"alma": [], "korte": {}}
if ("szar" in a.keys() and len(a['szar']) > 0):
  print("van szar")